In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('alb20-21data.txt')
winter2 = pd.read_csv ('alb21-22data.txt')
winter3 = pd.read_csv ('alb22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GS','IC','PL','SG','SN']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
precip_days2

,station,datetime,precip(mm),wxcodes
3,ALB,2021-12-01 03:49:00,T,-SN
4,ALB,2021-12-01 03:51:00,T,-SN
5,ALB,2021-12-01 04:51:00,T,-SN BR
6,ALB,2021-12-01 05:00:00,0.00,-SN BR
7,ALB,2021-12-01 05:51:00,0.25,-SN
...,...,...,...,...
2901,ALB,2022-02-27 22:36:00,T,-SN
2902,ALB,2022-02-27 22:51:00,T,-SN
2903,ALB,2022-02-27 22:58:00,T,-SN
2904,ALB,2022-02-27 23:11:00,0.00,-SN


In [6]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_496\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_496\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_496\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [7]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_496\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_496\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_496\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [8]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 51]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 51]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 51]

In [9]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [10]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [11]:
albany = LocationInfo(
    'Albany', 'New York', 
    latitude=42.6526, 
    longitude=-73.7562, 
    timezone='America/New_York'
)

albany

LocationInfo(name='Albany', region='New York', timezone='America/New_York', latitude=42.6526, longitude=-73.7562)

In [12]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(albany.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [13]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(albany.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [14]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(albany.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [15]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 07:06:16  2020-12-01 16:22:04  2020-12-01
1   2020-12-02 07:07:19  2020-12-02 16:21:49  2020-12-02
2   2020-12-03 07:08:22  2020-12-03 16:21:35  2020-12-03
3   2020-12-04 07:09:22  2020-12-04 16:21:24  2020-12-04
4   2020-12-05 07:10:22  2020-12-05 16:21:15  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:38:25  2021-02-24 17:38:31  2021-02-24
86  2021-02-25 06:36:51  2021-02-25 17:39:46  2021-02-25
87  2021-02-26 06:35:16  2021-02-26 17:41:01  2021-02-26
88  2021-02-27 06:33:40  2021-02-27 17:42:15  2021-02-27
89  2021-02-28 06:32:04  2021-02-28 17:43:30  2021-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 07:06:01  2021-12-01 16:22:08  2021-12-01
1   2021-12-02 07:07:04  2021-12-02 16:21:52  2021-12-02
2   2021-12-03 07:08:07  2021-12-03 16:21:38  2021-12-03
3   2021-12-04 07:09:08  2021-12-04 16:21:26  2021-12-04
4   2021-12-05 07:10:08  2021-12-05 16:21:17  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:38:48  2022-02-24 17:38:13  2022-02-24
86  2022-02-25 06:37:14  2022-02-25 17:39:28  2022-02-25
87  2022-02-26 06:35:39  2022-02-26 17:40:43  2022-02-26
88  2022-02-27 06:34:04  2022-02-27 17:41:57  2022-02-27
89  2022-02-28 06:32:27  2022-02-28 17:43:12  2022-02-28

[90 rows x 3 columns]


In [17]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 07:05:45  2022-12-01 16:22:12  2022-12-01
1   2022-12-02 07:06:49  2022-12-02 16:21:55  2022-12-02
2   2022-12-03 07:07:52  2022-12-03 16:21:41  2022-12-03
3   2022-12-04 07:08:53  2022-12-04 16:21:28  2022-12-04
4   2022-12-05 07:09:53  2022-12-05 16:21:19  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:39:10  2023-02-24 17:37:54  2023-02-24
86  2023-02-25 06:37:37  2023-02-25 17:39:10  2023-02-25
87  2023-02-26 06:36:02  2023-02-26 17:40:25  2023-02-26
88  2023-02-27 06:34:27  2023-02-27 17:41:39  2023-02-27
89  2023-02-28 06:32:50  2023-02-28 17:42:54  2023-02-28

[90 rows x 3 columns]


In [18]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_496\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [19]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,ALB,2020-12-02 13:51:00,0.0001,-SN,2020-12-02,2020-12-02 07:07:19,2020-12-02 16:21:49
1,ALB,2020-12-02 17:51:00,0.0001,-SN,2020-12-02,2020-12-02 07:07:19,2020-12-02 16:21:49
2,ALB,2020-12-02 18:51:00,0.0001,-SN,2020-12-02,2020-12-02 07:07:19,2020-12-02 16:21:49
3,ALB,2020-12-02 19:51:00,0.0001,-SN,2020-12-02,2020-12-02 07:07:19,2020-12-02 16:21:49
4,ALB,2020-12-02 21:51:00,0.0001,-SN,2020-12-02,2020-12-02 07:07:19,2020-12-02 16:21:49
...,...,...,...,...,...,...,...
332,ALB,2021-02-27 09:51:00,0.0001,-SN,2021-02-27,2021-02-27 06:33:40,2021-02-27 17:42:15
333,ALB,2021-02-27 10:51:00,0.7600,-SN BR,2021-02-27,2021-02-27 06:33:40,2021-02-27 17:42:15
334,ALB,2021-02-27 11:51:00,0.0001,-SN,2021-02-27,2021-02-27 06:33:40,2021-02-27 17:42:15
335,ALB,2021-02-27 12:51:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:33:40,2021-02-27 17:42:15


In [20]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,ALB,2021-12-01 03:51:00,0.0001,-SN,2021-12-01,2021-12-01 07:06:01,2021-12-01 16:22:08
1,ALB,2021-12-01 04:51:00,0.0001,-SN BR,2021-12-01,2021-12-01 07:06:01,2021-12-01 16:22:08
2,ALB,2021-12-01 05:51:00,0.2500,-SN,2021-12-01,2021-12-01 07:06:01,2021-12-01 16:22:08
3,ALB,2021-12-03 14:51:00,0.0001,-SN,2021-12-03,2021-12-03 07:08:07,2021-12-03 16:21:38
4,ALB,2021-12-05 09:51:00,0.0001,-SN,2021-12-05,2021-12-05 07:10:08,2021-12-05 16:21:17
...,...,...,...,...,...,...,...
227,ALB,2022-02-25 21:51:00,0.0001,-SN,2022-02-25,2022-02-25 06:37:14,2022-02-25 17:39:28
228,ALB,2022-02-26 01:51:00,0.0001,-SN,2022-02-26,2022-02-26 06:35:39,2022-02-26 17:40:43
229,ALB,2022-02-26 02:51:00,0.0001,-SN,2022-02-26,2022-02-26 06:35:39,2022-02-26 17:40:43
230,ALB,2022-02-27 22:51:00,0.0001,-SN,2022-02-27,2022-02-27 06:34:04,2022-02-27 17:41:57


In [21]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,ALB,2022-12-01 03:51:00,0.0001,-SN,2022-12-01,2022-12-01 07:05:45,2022-12-01 16:22:12
1,ALB,2022-12-11 13:51:00,0.0001,-SN,2022-12-11,2022-12-11 07:15:23,2022-12-11 16:21:11
2,ALB,2022-12-11 14:51:00,0.0001,-SN BR,2022-12-11,2022-12-11 07:15:23,2022-12-11 16:21:11
3,ALB,2022-12-11 15:51:00,0.2500,-SN BR,2022-12-11,2022-12-11 07:15:23,2022-12-11 16:21:11
4,ALB,2022-12-11 16:51:00,0.5100,SN BR,2022-12-11,2022-12-11 07:15:23,2022-12-11 16:21:11
...,...,...,...,...,...,...,...
232,ALB,2023-02-27 00:51:00,0.0001,-SN,2023-02-27,2023-02-27 06:34:27,2023-02-27 17:41:39
233,ALB,2023-02-27 01:51:00,0.0001,-SN,2023-02-27,2023-02-27 06:34:27,2023-02-27 17:41:39
234,ALB,2023-02-27 02:51:00,0.0001,-SN,2023-02-27,2023-02-27 06:34:27,2023-02-27 17:41:39
235,ALB,2023-02-27 03:51:00,0.0001,-SN,2023-02-27,2023-02-27 06:34:27,2023-02-27 17:41:39


In [73]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,ALB,2020-12-02 13:51:00,0.0001,-SN,2020-12-02,2020-12-02 07:07:19,2020-12-02 16:21:49
9,ALB,2020-12-08 07:51:00,0.0001,-FZDZSN,2020-12-08,2020-12-08 07:13:11,2020-12-08 16:21:04
10,ALB,2020-12-08 09:51:00,0.0001,-SN,2020-12-08,2020-12-08 07:13:11,2020-12-08 16:21:04
11,ALB,2020-12-08 10:51:00,0.0001,-SN,2020-12-08,2020-12-08 07:13:11,2020-12-08 16:21:04
12,ALB,2020-12-08 11:51:00,0.0001,-SN,2020-12-08,2020-12-08 07:13:11,2020-12-08 16:21:04
...,...,...,...,...,...,...,...
332,ALB,2021-02-27 09:51:00,0.0001,-SN,2021-02-27,2021-02-27 06:33:40,2021-02-27 17:42:15
333,ALB,2021-02-27 10:51:00,0.7600,-SN BR,2021-02-27,2021-02-27 06:33:40,2021-02-27 17:42:15
334,ALB,2021-02-27 11:51:00,0.0001,-SN,2021-02-27,2021-02-27 06:33:40,2021-02-27 17:42:15
335,ALB,2021-02-27 12:51:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:33:40,2021-02-27 17:42:15


In [23]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_496\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_496\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_496\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [77]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,ALB,2020-12-02 13:51:00,0.0001,-SN,2020-12-02,2020-12-02 07:07:19,2020-12-02 16:21:49
9,ALB,2020-12-08 07:51:00,0.0001,-FZDZSN,2020-12-08,2020-12-08 07:13:11,2020-12-08 16:21:04
10,ALB,2020-12-08 09:51:00,0.0001,-SN,2020-12-08,2020-12-08 07:13:11,2020-12-08 16:21:04
11,ALB,2020-12-08 10:51:00,0.0001,-SN,2020-12-08,2020-12-08 07:13:11,2020-12-08 16:21:04
12,ALB,2020-12-08 11:51:00,0.0001,-SN,2020-12-08,2020-12-08 07:13:11,2020-12-08 16:21:04
...,...,...,...,...,...,...,...
332,ALB,2021-02-27 09:51:00,0.0001,-SN,2021-02-27,2021-02-27 06:33:40,2021-02-27 17:42:15
333,ALB,2021-02-27 10:51:00,0.7600,-SN BR,2021-02-27,2021-02-27 06:33:40,2021-02-27 17:42:15
334,ALB,2021-02-27 11:51:00,0.0001,-SN,2021-02-27,2021-02-27 06:33:40,2021-02-27 17:42:15
335,ALB,2021-02-27 12:51:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:33:40,2021-02-27 17:42:15


In [25]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [26]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [27]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.4363261682242991
2021-2022: 2.1538417721518988
2022-2023: 0.4607488372093023


In [28]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
1,ALB,2020-12-02 17:51:00,0.0001,-SN,2020-12-02,2020-12-02 07:07:19,2020-12-02 16:21:49
2,ALB,2020-12-02 18:51:00,0.0001,-SN,2020-12-02,2020-12-02 07:07:19,2020-12-02 16:21:49
3,ALB,2020-12-02 19:51:00,0.0001,-SN,2020-12-02,2020-12-02 07:07:19,2020-12-02 16:21:49
4,ALB,2020-12-02 21:51:00,0.0001,-SN,2020-12-02,2020-12-02 07:07:19,2020-12-02 16:21:49
5,ALB,2020-12-08 02:51:00,0.0001,-SN,2020-12-08,2020-12-08 07:13:11,2020-12-08 16:21:04
...,...,...,...,...,...,...,...
326,ALB,2021-02-23 01:51:00,0.0001,-SN,2021-02-23,2021-02-23 06:39:58,2021-02-23 17:37:15
327,ALB,2021-02-23 05:51:00,0.0001,-SN,2021-02-23,2021-02-23 06:39:58,2021-02-23 17:37:15
328,ALB,2021-02-23 17:51:00,0.0001,-SN,2021-02-23,2021-02-23 06:39:58,2021-02-23 17:37:15
329,ALB,2021-02-23 18:51:00,0.0001,-SNRA BR,2021-02-23,2021-02-23 06:39:58,2021-02-23 17:37:15


In [29]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_496\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_496\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_496\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [30]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [31]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [32]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.16721318681318684
2021-2022: 0.5493436363636364
2022-2023: 0.3474289473684211
